In [1]:
%config Completer.use_jedi = False

import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# example of training an conditional gan on the fashion mnist dataset
from numpy import expand_dims
from numpy import zeros
from numpy import ones
import keras
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Reshape, multiply, Embedding, merge, Concatenate, Conv1D, BatchNormalization
from keras.layers import Dense, Flatten, Multiply, LSTM
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
from numpy import asarray
import matplotlib.pyplot as plt
from keras.layers import UpSampling1D
from keras.layers.core import Activation
from keras.layers import Add
import tensorflow as tf
from evaluation_metrics import *
metric_to_calculate = ['FID', 'MMD', 'DTW', 'PC', 'RMSE', 'TWED']
from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek, SMOTEENN

In [2]:
def reshape(X):
    if len(X.shape) == 1:
        X = X.reshape(X.shape[0], 1)
        return X
    else:
        if X.shape[-1] == 1:
            return X
        else:
            X = X.reshape(X.shape[0], X.shape[1], 1)
            return X

In [3]:
X = np.load('Data/Original/train_data.npy')
print (X.shape)
y = X[:,-1]
print (y.shape)
X = X[:,:-1]
print (X.shape)
# print (np.unique(y, return_counts=True))
# y[y==0].shape
# y==0

(204775, 187)
(204775,)
(204775, 186)


In [4]:
X, X_val, y, y_val = train_test_split(X, y, test_size=0.2)
print (X.shape, X_val.shape, y.shape, y_val.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_val, return_counts=True))

(163820, 186) (40955, 186) (163820,) (40955,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([31181,  2358,  7416]))


In [5]:
np.save('Data/TrainVal/X', X)
np.save('Data/TrainVal/y', y)
np.save('Data/TrainVal/X_val', X_val)
np.save('Data/TrainVal/y_val', y_val)

In [6]:
oversample = SMOTETomek(n_jobs=-1)
X_Aug, y_Aug = oversample.fit_resample(X, y)

np.save('Data/SMOTETOMEK/X', X_Aug)
np.save('Data/SMOTETOMEK/y', y_Aug)

print (X_Aug.shape, y_Aug.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_Aug, return_counts=True))

(376663, 186) (376663,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([125551, 125559, 125553]))


In [7]:
oversample = SMOTEENN(n_jobs=-1)
X_Aug, y_Aug = oversample.fit_resample(X, y)

np.save('Data/SMOTEENN/X', X_Aug)
np.save('Data/SMOTEENN/y', y_Aug)

print (X_Aug.shape, y_Aug.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_Aug, return_counts=True))

(373544, 186) (373544,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([122941, 125410, 125193]))


In [8]:
oversample = SMOTE(n_jobs=-1)
X_Aug, y_Aug = oversample.fit_resample(X, y)

np.save('Data/SMOTE/X', X_Aug)
np.save('Data/SMOTE/y', y_Aug)

print (X_Aug.shape, y_Aug.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_Aug, return_counts=True))

(376683, 186) (376683,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([125561, 125561, 125561]))


In [9]:
oversample = ADASYN(n_jobs=-1)
X_Aug, y_Aug = oversample.fit_resample(X, y)

np.save('Data/ADASYN/X', X_Aug)
np.save('Data/ADASYN/y', y_Aug)

print (X_Aug.shape, y_Aug.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_Aug, return_counts=True))

(377133, 186) (377133,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([125561, 125895, 125677]))


In [10]:
oversample = SVMSMOTE(n_jobs=-1)
X_Aug, y_Aug = oversample.fit_resample(X, y)

np.save('Data/SVMSMOTE/X', X_Aug)
np.save('Data/SVMSMOTE/y', y_Aug)

print (X_Aug.shape, y_Aug.shape)
print (np.unique(y, return_counts=True))
print (np.unique(y_Aug, return_counts=True))

(376683, 186) (376683,)
(array([0., 1., 2.]), array([125561,   9305,  28954]))
(array([0., 1., 2.]), array([125561, 125561, 125561]))


In [18]:
from helper import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [11]:
WINDOW_SIZE = 186
INPUT_FEAT = 1
OUTPUT_CLASS = 3

In [12]:
model = parallel_NN_LSTM(WINDOW_SIZE,INPUT_FEAT,OUTPUT_CLASS)
# model.summary()

In [18]:
print (y_test.shape, X_test.shape, y.shape, X.shape)

(40955,) (40955, 186) (163820,) (163820, 186)


In [14]:
cb = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5), 
      ModelCheckpoint('parallel_NN_LSTM.h5', save_best_only=True, monitor='val_loss', mode='min')]
history = model.fit(reshape(X), to_categorical(y), validation_data=(reshape(X_val), to_categorical(y_val)), 
                    batch_size=32, epochs=1, callbacks=cb, verbose=1)

5120/5120 [==============================] - 90s 17ms/step - loss: 0.1439 - accuracy: 0.9579 - val_loss: 0.0757 - val_accuracy: 0.9791


In [16]:
X_test = np.load('Data/test_data.npy')
print (X_test.shape)
y_test = X_test[:,-1]
print (y.shape)
X_test = X_test[:,:-1]
print (X_test.shape)

(203187, 187)
(131056,)
(203187, 186)


In [20]:
filename = 'parallel_NN_LSTM'
if not os.path.exists(str(filename)):
    os.makedirs(str(filename))
prediction(model, reshape(X_test), to_categorical(y_test), filename)
save_the_model(filename, model)
write_history(filename, history.history)

6350/6350 [==============================] - 17s 3ms/step


In [21]:
save_the_model(filename, model)
write_history(filename, history.history)

Saved model to disk


In [17]:
Parallel_GAP(WINDOW_SIZE, INPUT_FEAT, OUTPUT_CLASS)

(163820, 186, 1)

In [ ]:
X_Aug = np.load('Data/X_smote.npy')
y_Aug = np.load('Data/y_smote.npy')
X_Aug = np.load('Data/X_adasyn.npy')
y_Aug = np.load('Data/y_adasyn.npy')
X_Aug = np.load('Data/X_svmsmote.npy')
y_Aug = np.load('Data/y_svmsmote.npy')

In [ ]:
# temp_i = 0
# prev = 100
# for i, label in enumerate(y_smote):
#     if label != prev:
#         # print (i, label, prev)
#         print (i-temp_i, int(prev), int(label))
#         temp_i = i
#     prev = label